In [102]:
import string
import numpy as np

In [103]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    embedding = np.zeros(n_classes)
    embedding[idx] = 1.0
    return embedding


def word2seq(word: str, alphabet: list) -> np.ndarray:
    one_hots = [one_hot(len(alphabet), alphabet.index(c)) for c in word]
    return np.array(one_hots)


def seq2word(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(x)] for x in seq])

In [104]:
alphabet = string.ascii_letters + ' ' + string.punctuation + string.digits
n_symbols = len(alphabet)
print(n_symbols, alphabet)

95 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [105]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

### Define model

In [122]:
class RNN:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)

        self.W: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.b: np.ndarray = np.zeros(n_out)
    
    @property
    def parameters(self) -> tuple[np.ndarray]:
        return (self.U, self.V, self.d, self.W, self.b)
    
    def many2one(self, sequence: np.ndarray) -> tuple[np.ndarray]:
        context: np.ndarray = np.zeros(len(self.V))
        for x in sequence:
            context = np.tanh(self.U @ x + self.V @ context + self.d)

        output = softmax(self.W @ context + self.b)
        return (output, context)

    def one2many(self, x: np.ndarray, t: int) -> tuple[np.ndarray]:
        outputs: np.ndarray = np.zeros((t, len(self.W)))
        context = np.tanh(self.U @ x + self.d)

        for i in range(t):
            outputs[i] = softmax(self.W @ context + self.b)
            context = np.tanh(self.U @ outputs[i] + self.V @ context + self.d)

        return (outputs, context)

    def many2many(self, sequence: np.ndarray, t: int) -> tuple[np.ndarray]:
        outputs: np.ndarray = np.zeros((t, len(self.W)))
        context: np.ndarray = np.zeros(len(self.V))

        for i,x in enumerate(sequence):
            context = np.tanh(self.U @ x + self.V @ context + self.d)
        
        for i in range(t):
            outputs[i] = softmax(self.W @ context + self.b)
            context = np.tanh(self.U @ x + self.V @ context + self.d)
        
        return (outputs, context)


### Create and run model

In [123]:
n_in = n_symbols
n_h = 32
n_out = n_symbols

model = RNN(n_in, n_h, n_out)

In [124]:
text = 'the quick brown fox jumps over the lazy dog'
seq = word2seq(text, alphabet)

outputs, context = model.many2one(seq)
print(alphabet[np.argmax(outputs)])

U


In [125]:
text = 'a'
seq = word2seq(text, alphabet)

outputs, context = model.one2many(seq[0], 128)
print(seq2word(outputs, alphabet))

[;J&+V/6cN<U4wq S;U}w`w#dU8U#i`a=/cY:cdW<wX^6y@}8Ja|`=B@lm`aE =C"Ue|  )CYU"w|6yhWwG~~cBU88{s7:fw72,YU/8e?Jagl9u W;UUid_}FXw%`u+~


In [126]:
text = 'the quick brown fox jumps over the lazy dog'
seq = word2seq(text, alphabet)

outputs, context = model.many2many(seq, 128)
print(seq2word(outputs, alphabet))

UEwiMXJD`a=hi+w`*B7hh8LA!-7r`KY*`c;jUwydH7`.y=}M+&!DB`!,RU"D`OyCU-J`BL:j~|qMNCUJw| NdUUXJ`pB7x8JBz"R+wXw^*u,j+wXwp-,}:"81?>K4!` 
